In [1]:
import pandas as pd
import traceback
import OpenDartReader
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'
dart = OpenDartReader(api_key)

In [2]:
def str2num(x):
    if "," in x:
        x = x.replace(",","")
    if x.startswith("(") and x.endswith(")"):
        x = x.lstrip("(")
        x = x.rstrip(")")
        float_num = float(x) * (-1)
    else:
        x = x.lstrip("(")
        x = x.rstrip(")")     
        float_num = float(x)
    return float_num

In [8]:
reports = dart.list(start='2017-04-01', end='2017-06-30', kind='A', final=True)

In [9]:
# type(reports)
reports.shape

(2670, 9)

In [10]:
reports.head(5)

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00216027,바이넥스,053030,K,[기재정정]분기보고서 (2017.03),20170630000550,바이넥스,20170630,
1,00216027,바이넥스,053030,K,[기재정정]사업보고서 (2016.12),20170630000543,바이넥스,20170630,연
2,00216027,바이넥스,053030,K,[기재정정]사업보고서 (2015.12),20170630000535,바이넥스,20170630,
3,00216027,바이넥스,053030,K,[기재정정]사업보고서 (2014.12),20170630000526,바이넥스,20170630,
4,00216027,바이넥스,053030,K,[기재정정]사업보고서 (2013.12),20170630000518,바이넥스,20170630,연


In [11]:
str_expr = "report_nm.str.contains('감사보고서')"
biz_repports = reports.query(str_expr)

In [13]:
biz_repports

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm


In [10]:
str_expr = "report_nm.str.contains('사업보고서')"
biz_repports = reports.query(str_expr)

In [11]:
biz_repports.shape

(99, 9)

In [12]:
subreports = []
str_expr = "title.str.contains('5. 재무제표 주석')"
for i, row in biz_repports.iterrows():
    try:
        rpt_name = row['report_nm']
        rpt_num = row['rcept_no']        
        rpt_url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + rpt_num
        df_rpt = dart.sub_docs(rpt_num)
        rpt_data = df_rpt.query(str_expr)       # data frame
        if rpt_data.size == 0:
            subreports.append({'rpt_num' : rpt_num, 'corp_name' : row['corp_name'], 'stock_code' : row['stock_code'], 'rpt_name' : rpt_name, 'rpt_url' : rpt_url,
                            'rpt_title' : None, 'rpt_notes_url' : None, 'result' : 'No Financial Statement Notes'})                 
        else:
            rpt_title = rpt_data.iloc[0]['title']
            rpt_notes_url = rpt_data.iloc[0]['url']
            subreports.append({'rpt_num' : rpt_num, 'corp_name' : row['corp_name'], 'stock_code' : row['stock_code'], 'rpt_name' : rpt_name, 'rpt_url' : rpt_url,
                            'rpt_title' : rpt_title, 'rpt_notes_url' : rpt_notes_url, 'result' : None}) 
    except Exception as e:
        print(row['rcept_no'])
        print(row['stock_code'])
        print(row['report_nm'])
        print('http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + row['rcept_no'])
        err_msg = traceback.format_exc()
        print(err_msg)

In [13]:
len(subreports)

99

In [45]:
data_list = []
for r in subreports:
    try:
        val_a = val_b = None
        url = r['rpt_notes_url']
        if url == None:
            data_list.append({'corp_name' : r['corp_name'], 'stock_code' : r['stock_code'], 'rpt_num' : r['rpt_num'], 'rpt_name' : r['rpt_name'], 
                            'val_a' : None, 'val_b' : None, 'result' : 'No Financial Statement Notes'})
        else:
            doc_data = pd.read_html(url)
            for df in doc_data:
                row = df.shape[0]
                for i in range(0, row):
                    item = str(df.iloc[i,0])
                    if item.__contains__('확정급여채무의 현재가치'):
                        # print(i, type(df.iloc[i,1]), df.iloc[i,1])
                        str_a = str(df.iloc[i,1])
                        if str_a == "-": val_a = None
                        else: val_a = str2num(str_a)
                    elif item.__contains__('사외적립자산의 공정가치'):
                        # print(i, type(df.iloc[i,1]), df.iloc[i,1])                            
                        str_b = str(df.iloc[i,1])
                        if str_b == "-": val_b = None
                        else: val_b = str2num(str_b)
                    else:
                        continue    
                if (val_a != None) and (val_b != None):
                    break          
            data_list.append({'corp_name' : r['corp_name'], 'stock_code' : r['stock_code'], 'rpt_num' : r['rpt_num'], 'rpt_name' : r['rpt_name'], 
                            'val_a' : val_a, 'val_b' : val_b, 'result' : 'OK'})
    except ValueError as err:
        print(r['rpt_num'])
        # print(r['rpt_url'])
        print(r['rpt_notes_url'])
        # print(err)
        data_list.append({'corp_name' : r['corp_name'], 'stock_code' : r['stock_code'], 'rpt_num' : r['rpt_num'], 'rpt_name' : r['rpt_name'], 
                        'val_a' : val_a, 'val_b' : val_b, 'result' : str(err)})        
    except Exception as e:
        print(r['rpt_num'])
        # print(r['rpt_url'])
        print(r['rpt_notes_url'])
        err_msg = traceback.format_exc()
        print(err_msg)

20170428002043
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170428002043&dcmNo=5617894&eleId=16&offset=555569&length=130&dtd=dart3.xsd
20170428001871
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170428001871&dcmNo=5617406&eleId=16&offset=608249&length=216&dtd=dart3.xsd
20170428001837
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170428001837&dcmNo=5617323&eleId=16&offset=572391&length=211&dtd=dart3.xsd
20170428001583
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170428001583&dcmNo=5616748&eleId=16&offset=741910&length=136&dtd=dart3.xsd
20170428001099
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170428001099&dcmNo=5616006&eleId=16&offset=799597&length=196&dtd=dart3.xsd
20170427000709
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170427000709&dcmNo=5612910&eleId=16&offset=636285&length=229&dtd=dart3.xsd
20170418000281
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170418000281&dcmNo=5600517&eleId=16&offset=56844&length=121&dtd=dart3.xsd
20170410002581
http://dart.fss.or.kr/repor

In [46]:
len(data_list)

99

In [47]:
df_all = pd.DataFrame(data_list)

In [48]:
df_all.shape

(99, 7)

In [49]:
df_all['result'].value_counts()

result
OK                              79
No Financial Statement Notes    11
No tables found                  9
Name: count, dtype: int64

In [50]:
df_all[df_all['result'] == 'OK']

,corp_name,stock_code,rpt_num,rpt_name,val_a,val_b,result
3,알바이오,003190,20170428001738,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
4,세종,,20170428001652,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
6,이에스산업,241510,20170428001415,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
8,듀켐바이오,176750,20170428001089,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
9,에스텍시스템,,20170428000888,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
...,...,...,...,...,...,...,...
94,크린앤사이언스,045520,20170403000445,[기재정정]사업보고서 (2016.12),3885459.0,-3841856.0,OK
95,피엠티,147760,20170403000323,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
96,퓨쳐켐,220100,20170403000225,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
97,교보증권,030610,20170403000043,[기재정정]사업보고서 (2016.12),30458251.0,-30339335.0,OK


In [51]:
df_all[df_all['result'] == 'No Financial Statement Notes']

,corp_name,stock_code,rpt_num,rpt_name,val_a,val_b,result
11,비앤에스미디어,156170,20170428000153,[첨부정정]사업보고서 (2016.12),NaN,NaN,No Financial Statement Notes
14,큐렉소,060280,20170426000359,[첨부정정]사업보고서 (2016.12),NaN,NaN,No Financial Statement Notes
16,삼일,032280,20170426000246,[기재정정]사업보고서 (2012.12),NaN,NaN,No Financial Statement Notes
18,우진,105840,20170421000696,[기재정정]사업보고서 (2013.12),NaN,NaN,No Financial Statement Notes
19,우양에이치씨,101970,20170421000455,[첨부정정]사업보고서 (2016.12),NaN,NaN,No Financial Statement Notes
25,한화,000880,20170419000057,[첨부정정]사업보고서 (2016.12),NaN,NaN,No Financial Statement Notes
45,한창산업,079170,20170411000156,[첨부정정]사업보고서 (2016.12),NaN,NaN,No Financial Statement Notes
47,넵튠,217270,20170410002533,[첨부정정]사업보고서 (2016.12),NaN,NaN,No Financial Statement Notes
57,롯데손해보험,000400,20170406002062,[첨부정정]사업보고서 (2016.12),NaN,NaN,No Financial Statement Notes
59,농업회사법인합천유통,,20170406001175,[첨부정정]사업보고서 (2016.12),NaN,NaN,No Financial Statement Notes


In [52]:
df_all[df_all['result'] == 'No tables found']

,corp_name,stock_code,rpt_num,rpt_name,val_a,val_b,result
0,이스트아시아홀딩스,900110,20170428002043,사업보고서 (2016.12),NaN,NaN,No tables found
1,오가닉티코스메틱,900300,20170428001871,사업보고서 (2016.12),NaN,NaN,No tables found
2,차이나그레이트스타인터내셔널리미티드,900040,20170428001837,사업보고서 (2016.12),NaN,NaN,No tables found
5,로스웰,900260,20170428001583,사업보고서 (2016.12),NaN,NaN,No tables found
7,엘브이엠씨,900140,20170428001099,사업보고서 (2016.12),NaN,NaN,No tables found
13,골든센츄리,900280,20170427000709,사업보고서 (2016.12),NaN,NaN,No tables found
28,베어리버,,20170418000281,사업보고서 (2016.12),NaN,NaN,No tables found
46,뉴아세아,013340,20170410002581,사업보고서 (2016.12),NaN,NaN,No tables found
61,마이크로사이언스테크,,20170406000945,사업보고서 (2016.12),NaN,NaN,No tables found


In [20]:
df_all.dtypes

corp_name      object
stock_code     object
rpt_num        object
rpt_name       object
val_a         float64
val_b         float64
result         object
dtype: object

In [34]:
df_all

,corp_name,stock_code,rpt_num,rpt_name,val_a,val_b,result
0,이스트아시아홀딩스,900110,20170428002043,사업보고서 (2016.12),NaN,NaN,No tables found
1,오가닉티코스메틱,900300,20170428001871,사업보고서 (2016.12),NaN,NaN,No tables found
2,차이나그레이트스타인터내셔널리미티드,900040,20170428001837,사업보고서 (2016.12),NaN,NaN,No tables found
3,알바이오,003190,20170428001738,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
4,세종,,20170428001652,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
...,...,...,...,...,...,...,...
94,크린앤사이언스,045520,20170403000445,[기재정정]사업보고서 (2016.12),3885459.0,-3841856.0,OK
95,피엠티,147760,20170403000323,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
96,퓨쳐켐,220100,20170403000225,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
97,교보증권,030610,20170403000043,[기재정정]사업보고서 (2016.12),30458251.0,-30339335.0,OK


In [ ]:
def year_month(str):
    substr = str[-8:-1] 
    y, m = substr.split(".")
    return y, m

df_all['year'], df_all['month'] = zip(*df_all['rpt_name'].apply(year_month))

In [53]:
df_all.to_csv('D:/PythonProject/data-gatherer/dart_fs_notes/output/result_1m.csv')